# Visualise *Ergo* data
This notebook contains various ad-hoc means of visualising data collected from *Ergo*. See the report [here](https://git.cs.sun.ac.za/Computer-Science/rw771/2022/26723077-TG7-doc) or the source code [here](https://git.cs.sun.ac.za/Computer-Science/rw771/2022/26723077-TG7-src).

## Import libraries and define constants

In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
import matplotlib as mpl
# By default use a larger figure size
mpl.rcParams['figure.figsize'] = [12, 12]
mpl.rcParams['figure.dpi'] = 72
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
# Setup seaborn
sns.set()
%matplotlib inline

# Define some variables for creating feature names like 'left-thumb-x' or 'right-index-z'
hands = ['left', 'right']
finger_names = ['thumb', 'index', 'middle', 'ring', 'little']
dimensions = ['x', 'y', 'z']
# Actually create some feature names to give the data meaningful labels
fingers = []
for hand in hands:
    for finger_name in finger_names:
        for dimension in dimensions:
            fingers.append(f'{hand}-{finger_name}-{dimension}')

## Interactively view the raw data
This cell lets you view the raw training data as found in `../gesture_data/train/{gesture}/{timestamp}`. Run the cell, and then you can choose which gesture to view (ie `gesture0001`, `gesture0002`, `gesture0003`) and then which observation of that gesture to view (ie `1`, `2`, `3`). 

Note that the raw sensor data is not all centered at the same position (so a no-acceleration value from one sensor might be different to the no-acceleration value from another sensor). Check the `normalise` checkbox in order to roughly center all the values around their common average. Specifically, the `-z` sensors are usually quite different from the `-x` and `-y` sensors since the `-z` component is in the direction of gravity.

In [11]:
# Get a listing of all directories and their files
dir_files = {d: os.listdir(f'../gesture_data/train/{d}') for d in os.listdir(f'../gesture_data/train') if d != ".DS_Store"}
# Filter out all directories which don't have any files in them
dir_files = {d: files for d, files in dir_files.items() if len(files) > 0}
max_files = max([len(files) for files in dir_files.values()])
dirs = sorted(list(dir_files.keys()))

@interact(gesture=dirs, idx=(0, max_files, 1), normalise=True)
def showfile(gesture, idx, normalise):
    try:
        names = [f for f in os.listdir(f'../gesture_data/train/{gesture}') if f.endswith('.txt')]   
        idx = min(idx, len(names)-1)
        if idx == len(names)-1:
            print(f"""Warning: you've selected an index ({idx}) greater than the number of observations for gesture {gesture} () so observation {len(names)-1} is being shown instead.""")
        filename = f'../gesture_data/train/{gesture}/{names[idx]}'
        arr = np.genfromtxt(filename, delimiter=",")[:,1:]
    except FileNotFoundError:
        print(f"Failed to find file for {gesture=}, {idx=}")
        return
    if normalise:
        arr = arr - np.mean(arr, axis=0) + np.mean(arr)
    arr = arr.T
    cumsum = np.cumsum(arr - np.repeat(np.mean(arr, axis=1), arr.shape[1]).reshape(arr.shape), axis=1)

    fig, axs = plt.subplots(1, figsize=(16,10))

    fig.suptitle(f'File {filename}')
    
    # Plot the heatmap
    img = axs.imshow(arr, cmap='viridis', aspect='auto')

    # Plot line chart 1
    axs.set_yticks([i for i in range(30)])
    axs.set_yticklabels(fingers)
    axs.grid(visible=None)
    

interactive(children=(Dropdown(description='gesture', options=('gesture0002', 'gesture0004', 'gesture0012', 'g…